In [1]:
import pandas as pd
import numpy as np
import re
from kiwipiepy import Kiwi
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import pymysql


In [2]:
con = pymysql.connect(
    host='localhost',
    user='root',
    password='1111',
    db='yorehalre',
    charset='utf8'
)

data = pd.read_sql("SELECT recipeID, recipe_name, summary, ingredient_name, category FROM recipe_info", con)
data_re = pd.read_sql("SELECT category, recipeID, recipe_name, user_id, rating, review_content FROM user_review", con)

con.close()

C:\Users\SSO\AppData\Local\Temp\ipykernel_24420\2245140572.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql("SELECT recipeID, recipe_name, summary, ingredient_name, category FROM recipe_info", con)
C:\Users\SSO\AppData\Local\Temp\ipykernel_24420\2245140572.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_re = pd.read_sql("SELECT category, recipeID, recipe_name, user_id, rating, review_content FROM user_review", con)


In [3]:
# data = pd.read_csv('../data/weaviate_data(임시).csv')
# data_re = pd.read_csv('../data/user_review.csv')

In [4]:
source = []
for ingredient_name in data['ingredient_name']:
    ingredient_name_str = str(ingredient_name)
    if '양념' in ingredient_name_str:
        ingredient_matches = re.findall(r'\[.*?\](.*?)\양념', ingredient_name_str)
        source_per_ingredient = []
        for ingredient_match in ingredient_matches:
            ingredient_cleaned = re.sub(r'[0-9]+[가-힣]+', '', ingredient_match).strip().split()
            for ing in ingredient_cleaned:
                if ing:
                    ingredient_final = re.sub(r'[^가-힣\s]', '', ing).strip()
                    if ingredient_final:
                        source_per_ingredient.extend(ingredient_final.split())
    else:
        ingredient_matches = re.findall(r'\[.*?\](.*)', ingredient_name_str)
        source_per_ingredient = []
        for ingredient_match in ingredient_matches:
            ingredient_cleaned = re.sub(r'[0-9]+[가-힣]+', '', ingredient_match).strip().split()
            for ing in ingredient_cleaned:
                if ing:
                    ingredient_final = re.sub(r'[^가-힣\s]', '', ing).strip()
                    if ingredient_final:
                        source_per_ingredient.extend(ingredient_final.split())
    # if source_per_ingredient:
    source.append(source_per_ingredient)

In [5]:
len(source)

874

In [6]:
sum(1 for item in source if not item)

32

In [7]:
data['main_source'] = source

In [8]:
data_re.head(3)

,category,recipeID,recipe_name,user_id,rating,review_content
0,간 건강,765,카레 해물 파스타,3366,3,참고해서 잘 해먹었습니다~ 우유가 없어서 카레를 그냥 재료들과 볶았는데 나쁘지 않았어요
1,간 건강,768,생생정보 오이미역냉국 황금레시피,1013,5,애들이 맛있다고 잘먹어요..
2,간 건강,768,생생정보 오이미역냉국 황금레시피,1173,5,정말 맛나요~


In [9]:
df = pd.merge(data, data_re, on=['recipeID', 'recipe_name', 'category'], how='outer')

In [10]:
kiwi = Kiwi()
def extract_keywords(text):
    stop_words = ['물', '약간', '적당', '적당량', '조금', '때', '레시피', '요리', '저', '거', '것', '수', '나']
    keywords = []
    text = text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    for title in text:
        titles = []
        result = kiwi.analyze(title)
        for token, tag, _ , _ in result[0][0]:
            if tag[0] in 'NN':
                if token not in stop_words and not re.match(r'.*\W.*', token):
                    titles.append(token.strip())
        keywords.append(titles)
    return keywords

In [11]:
name = extract_keywords(df['recipe_name'].apply(str))
summary = extract_keywords(df['summary'].apply(str))
source = extract_keywords(df['main_source'].apply(str))

In [12]:
review = extract_keywords(df['review_content'].apply(str))


In [13]:
list_all = []

for item_1, item_2, item_3, item_4 in zip(name, summary, df['main_source'], review):
    combined_list = item_1 + item_2 + item_3 + item_4
    list_all.append(combined_list)

In [14]:
df['all'] = list_all

In [15]:
all_keyword = []
for i in df['all']:
    for item in i:
        all_keyword.append(item)
print(len(all_keyword))

86131


In [16]:
unique_keyword = set(all_keyword)
# print(unique_keyword)
print(len(unique_keyword))


4022


In [17]:
'우엉' in unique_keyword

True

In [18]:
total_recipe_count = len(set(data['recipe_name']))
keyword_count_dict = dict.fromkeys(unique_keyword)

for keyword_list in df['all']:
    for keyword in keyword_list:
        if keyword_count_dict[keyword.strip()] == None:
            keyword_count_dict[keyword.strip()] = 1
        else:
            keyword_count_dict[keyword.strip()] += 1

In [19]:
keyword_idf = dict()
for each_keyword in keyword_count_dict:
    keyword_idf[each_keyword] = np.log10(total_recipe_count/keyword_count_dict[each_keyword])

In [20]:
sorted_data_dict = dict(sorted(keyword_idf.items(), key=lambda item: item[1], reverse=True))

In [21]:
all_category = data['category'].explode().tolist()
all_keyword = df['all'].explode().tolist()
unique_keyword = set(all_keyword)
unique_category = set(all_category)

In [22]:
unique_category

{'간 건강',
 '골다공증',
 '노화방지',
 '머슬업 다이어트',
 '변비',
 '스트레스 해소',
 '암 예방',
 '위 건강',
 '장 건강',
 '저칼로리 다이어트',
 '치매예방',
 '피로회복',
 '혈당조절',
 '혈액순환',
 '호흡기 건강'}

## category 별 데이터프레임

In [23]:
category_representation = pd.DataFrame(columns=sorted(unique_category), index=df['recipeID'])
print(category_representation.head(1))

         간 건강 골다공증 노화방지 머슬업 다이어트   변비 스트레스 해소 암 예방 위 건강 장 건강 저칼로리 다이어트 치매예방  \
recipeID                                                                      
0         NaN  NaN  NaN      NaN  NaN     NaN  NaN  NaN  NaN       NaN  NaN   

         피로회복 혈당조절 혈액순환 호흡기 건강  
recipeID                        
0         NaN  NaN  NaN    NaN  


In [124]:
for category in df['category']:
    # 해당하는 카테고리에는 1을 넣고 아니면 0을 넣음
    category_representation[category] = np.where(df['category'] == category, 3, None)

In [125]:
print(category_representation.tail(5))

          간 건강  골다공증  노화방지 머슬업 다이어트    변비 스트레스 해소  암 예방  위 건강  장 건강 저칼로리 다이어트  \
recipeID                                                                        
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
873       None  None  None     None  None    None  None  None  None      None   

         치매예방  피로회복  혈당조절  혈액순환 호흡기 건강  
recipeID                                
872         3  None  None  None   None  
872         3  None  None  None   None  
872         3  None  None  None   None  
872         3  None  None  None   None  
873         3  None  None  None   None  


## keyword 별 데이터프레임

In [126]:
keyword_representation = pd.DataFrame(columns=sorted(unique_keyword), index=df['recipeID'])
print(keyword_representation.head(1))

          2손다  3스픈    ᆢ  ᆢ저는    ㅋ   ㅋ넘   ㅋ존    ㅎ   ㅡ넘    가  ...  흰우유   흰자  \
recipeID                                                    ...             
0         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   

           흰콩  흰후추 흰후추가루    히   히트   힌트   힐링    힘  
recipeID                                           
0         NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  

[1 rows x 4022 columns]


In [127]:
for index in tqdm(range(len(df['recipeID']))):
    keywords = df['all'][index]
    recipeID = df['recipeID'].iloc[index]
    for keyword in keywords:
        keyword_idf_value = keyword_idf.get(keyword.strip(), None) 
        if keyword_idf_value is not None:
            keyword_representation.at[recipeID, keyword] = keyword_idf_value

# keyword_representation.sort_index(inplace=True)

100%|██████████| 4025/4025 [00:11<00:00, 338.07it/s]


In [128]:
keyword_representation.head(5)

,2손다,3스픈,ᆢ,ᆢ저는,ㅋ,ㅋ넘,ㅋ존,ㅎ,ㅡ넘,가,...,흰우유,흰자,흰콩,흰후추,흰후추가루,히,히트,힌트,힐링,힘
recipeID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 카테고리 + 키워드

In [129]:
recipe_representation = pd.concat([category_representation, keyword_representation], axis=1).fillna(0)


In [130]:
recipe_representation = recipe_representation.drop_duplicates()
print(recipe_representation.shape)
print(recipe_representation.describe())

(871, 4037)
             간 건강        골다공증        노화방지    머슬업 다이어트          변비     스트레스 해소  \
count  871.000000  871.000000  871.000000  871.000000  871.000000  871.000000   
mean     0.206659    0.206659    0.199770    0.206659    0.196326    0.206659   
std      0.760219    0.760219    0.748362    0.760219    0.742339    0.760219   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
75%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
max      3.000000    3.000000    3.000000    3.000000    3.000000    3.000000   

             암 예방        위 건강        장 건강   저칼로리 다이어트  ...         흰우유  \
count  871.000000  871.000000  871.000000  871.000000  ...  871.000000   
mean     0.189437    0.206659    0.210103    0.206659  ...    0.005980   
std      0.730094    0.760219    0.7

In [131]:
def cos_sim_matrix(a, b):
    cos_sim = cosine_similarity(a, b)
    result_df = pd.DataFrame(data=cos_sim, index=[a.index])
    
    return result_df

In [132]:
cs_df = cos_sim_matrix(recipe_representation, recipe_representation)
cs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,861,862,863,864,865,866,867,868,869,870
recipeID,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.207620,0.198950,0.241390,0.227305,0.310914,0.187746,0.193299,0.369776,0.173060,...,0.000000,0.000158,0.000000,0.000000,0.000000,0.000098,0.000000,0.019749,0.013197,0.000000
1,0.207620,1.000000,0.166418,0.204524,0.192590,0.265160,0.159893,0.164853,0.315361,0.157230,...,0.000000,0.027339,0.000000,0.000000,0.000000,0.001590,0.002129,0.000000,0.001689,0.000000
2,0.198950,0.166418,1.000000,0.212371,0.284568,0.249214,0.150277,0.154954,0.296396,0.138717,...,0.000000,0.000008,0.000000,0.008241,0.000000,0.009993,0.000000,0.000000,0.000000,0.000043
3,0.241390,0.204524,0.212371,1.000000,0.223916,0.302376,0.182334,0.188008,0.359622,0.170479,...,0.000000,0.001418,0.000000,0.000030,0.000000,0.000870,0.002475,0.000000,0.001964,0.000026
4,0.227305,0.192590,0.284568,0.223916,1.000000,0.284733,0.171695,0.188865,0.340112,0.160624,...,0.000712,0.001832,0.010959,0.000000,0.000212,0.019169,0.034718,0.000000,0.002472,0.000025


In [133]:
cs_df.shape

(871, 871)

In [134]:
print(cs_df[1].sort_values(ascending=False)[1:4])

recipeID
18          0.340001
8           0.315361
28          0.301936
Name: 1, dtype: float64


In [136]:
print(data.loc[1])
print(data.loc[18])
print(data.loc[8])
print(data.loc[28])

recipeID                                                           1
recipe_name                                           아삭아삭 연근장아찌 만들기
summary            연근은 비타민C와 식이섬유소가 풍부하고 빈혈예방에 탁월하고 코피가 자주나는 아이들에...
ingredient_name    [재료] 연근  [양념비율(200ml 계량컵 기준)] 간장  1 물  1 설탕  0...
category                                                   저칼로리 다이어트
main_source                                                     [연근]
Name: 1, dtype: object
recipeID                                                     18
recipe_name                                          양파 장아찌 만들기
summary            제철에 담아두면 일 년 내내 맛있게 드실 수가 있어서 참 좋은 양파장아찌입니다.
ingredient_name                         [재료] 양파  간장  물  설탕  식초 
category                                              저칼로리 다이어트
main_source                                 [양파, 간장, 물, 설탕, 식초]
Name: 18, dtype: object
recipeID                                                           8
recipe_name                                                    곤약 무침
summary          

In [138]:
train_df, test_df = train_test_split(data_re, test_size=0.2, random_state=1234)

In [139]:
test_userids = list(set(test_df.user_id.values))

In [140]:
result_df = pd.DataFrame()

for user_id in tqdm(test_userids):
    user_record_df = train_df.loc[train_df.user_id == int(user_id), :]
    
    user_sim_df = cs_df.loc[user_record_df['recipeID']]
    user_rating_df = user_record_df[['rating']]

    sim_sum = np.sum(user_sim_df.T.to_numpy(), -1)     # 각 열의 값을 합산
    prediction = np.matmul(user_sim_df.T.to_numpy(), user_rating_df.to_numpy()).flatten() / (sim_sum+1)
    
    # 프린트 하는 과정
    prediction_df = pd.DataFrame(prediction, index=cs_df.index).reset_index()
    prediction_df.columns = ['recipeID', 'pred_rating']
    prediction_df = prediction_df[['recipeID', 'pred_rating']][prediction_df.recipeID.isin(test_df[test_df.user_id == user_id]['recipeID'])]
    
    temp_df = prediction_df.merge(test_df[test_df.user_id == user_id], on='recipeID')
    result_df = pd.concat([result_df, temp_df], axis=0)

100%|██████████| 695/695 [00:08<00:00, 84.70it/s] 


In [141]:
result_df.head(2)

,recipeID,pred_rating,category,recipe_name,user_id,rating,review_content
0,356,0.0,장 건강,"표고버섯덮밥, 덮밥요리 간단 레시피",240,4,간단하고 맛있어요~
0,869,0.0,치매예방,절대 후회없는 깻잎김치 황금레시피^^,1846,5,간단하고 맛있어서 밥반찬으로 너무 좋아요~^^


In [142]:
print(f"RMSE: {sqrt(mean_squared_error(result_df['rating'].values, result_df['pred_rating'].values))}")

RMSE: 4.869690033239285


# 협업필터링

## 아이템 기반

In [41]:
train_df, test_df = train_test_split(data_re, test_size=0.2, random_state=1234)

In [42]:
train_df.shape, test_df.shape

((2824, 6), (706, 6))

In [43]:
user_ids = sorted(set(train_df['user_id'].values))
recipe_ids = sorted(set(train_df['recipeID'].values))

print(f'유저 수: {len(user_ids)}, 레시피 수: {len(recipe_ids)}')


유저 수: 2645, 레시피 수: 351


In [44]:
sparse_matrix = pd.DataFrame(index=recipe_ids, columns=user_ids)
sparse_matrix

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
train_df.head(10)

,category,recipeID,recipe_name,user_id,rating,review_content
1348,스트레스 해소,376,불닭볶음면+틈새라면=지옥의 염라면 만들기,765,5,맛있네요!
3259,혈액순환,479,부추무침,1108,5,정말 맛있어요^^ 잘 이용하겠습니당
235,골다공증,197,통조림으로 만든 집밥 백선생 일본식 꽁치조림,62,5,매번 레시피 잊어버리는데 만 개의 레시피를 보고 다시 떠올리곤 합니다. ㅎ
671,머슬업 다이어트,84,고등어무조림,2087,5,레시피대로햇더니 맛나네요~~생강가루 조금 첨가했어요 울아찌 맛있다고 한그릇 뚝딱햇어요
89,간 건강,784,비오는날에는 ★새우부추전,2569,5,간단하게 할수 있고 영양 좋은 음식이라\r\n좋았읍니다.
2655,치매예방,869,절대 후회없는 깻잎김치 황금레시피^^,2083,5,색깔이며 맛이며 좋아요^^\r\n다만 깻잎두장씩 해서했는데도 양념이 계량된데로하니 ...
507,머슬업 다이어트,84,고등어무조림,1107,5,이대로만 하시면 엄마가 해주시던 고등어조림 맛이 날겁니다. \r\n저는 무가 푹 익...
29,간 건강,768,생생정보 오이미역냉국 황금레시피,733,5,미역오이냉국이예여\r\n함올려봣어여\r\n맛잇겟죠
218,골다공증,196,송일국 꽁치쌈장 만들기 ★,3219,5,강추! 너무 맛있어요. 상추쌈에도 굿~ 입니다. 감사해요.
1130,머슬업 다이어트,110,닭가슴살 갈비구이,3169,5,만들기 쉽고 맛있어요~ 저는 간장 4스푼 넣었구요. 닭만 먹으면 조금 짠데 밥이나 ...


In [46]:
train_df_grouped = train_df.groupby(['recipeID', 'user_id'])['rating'].mean().reset_index()
sparse_matrix = train_df_grouped.pivot(index='recipeID', columns='user_id', values='rating')
sparse_matrix.index.name = 'recipeID'
sparse_matrix

user_id,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
recipeID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
def cossim_matrix(a, b):
    cossim_values = cosine_similarity(a.values, b.values)
    cossim_df = pd.DataFrame(data=cossim_values, columns = a.index.values, index=a.index)

    return cossim_df

In [48]:
item_sparse_matrix = sparse_matrix.fillna(0)
item_sparse_matrix

user_id,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
recipeID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
item_cossim_df = cossim_matrix(item_sparse_matrix, item_sparse_matrix)
item_cossim_df

,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
recipeID,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.022421,0.000000,0.0
868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0
869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.042741,0.022421,0.0,1.000000,0.052497,0.0


In [50]:
def get_item_based_collabor(title):
    return item_cossim_df[title].sort_values(ascending=False)[:5]

In [51]:
get_item_based_collabor(863)

recipeID
863    1.000000
869    0.042741
1      0.000000
604    0.000000
617    0.000000
Name: 863, dtype: float64

In [52]:
print(data.loc[data['recipeID'] == 863, 'recipe_name'])
print(data.loc[data['recipeID'] == 869, 'recipe_name'])

863    초간단한 조리법, 개운한 양파 겉절이
Name: recipe_name, dtype: object
869    절대 후회없는 깻잎김치 황금레시피^^
Name: recipe_name, dtype: object


In [53]:
userId_grouped = train_df.groupby('user_id')
# userId_grouped.indices.keys() : user_id
# item_sparse_matrix.index : recipe_id
item_prediction_result_df = pd.DataFrame(index=list(userId_grouped.indices.keys()), columns=item_sparse_matrix.index)
item_prediction_result_df

recipeID,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
for user_id, group in tqdm(userId_grouped):
    user_sim = item_cossim_df.loc[group['recipeID']]   # 유저의 레시피 유사도
    user_rating = group['rating']   # 유저의 평점

    # 사용자가 평가한 레시피들의 유사도 총합을 계산
    sim_sum = user_sim.sum(axis=0)
    
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    item_prediction_result_df.loc[user_id] = pred_ratings

 12%|█▏        | 326/2645 [00:01<00:09, 234.85it/s]

100%|██████████| 2645/2645 [00:06<00:00, 381.43it/s]


In [55]:
item_prediction_result_df.head(10)

recipeID,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.137207,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.231965,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.104691,0.0,0.0
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.144901,0.0,0.0,0.214043,0.0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.227307,0.0,0.0,0.0,0.0
1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.204947,0.109648,0.0,2.5,0.249394,0.0


In [56]:
print(item_prediction_result_df.shape)

(2645, 351)


## 유저 기반

In [57]:
user_sparse_matrix = sparse_matrix.fillna(0).transpose()
user_sparse_matrix.head(5)

recipeID,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
user_sparse_matrix.shape

(2645, 351)

In [59]:
user_cossim_df = cossim_matrix(user_sparse_matrix, user_sparse_matrix)
user_cossim_df

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [60]:
recipeId_grouped = train_df.groupby('recipeID')
user_prediction_result_df = pd.DataFrame(index=list(recipeId_grouped.indices.keys()), columns=user_sparse_matrix.index)
user_prediction_result_df

user_id,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
for recipeID, group in tqdm(recipeId_grouped):
    user_sim = user_cossim_df.loc[group['user_id']]
    user_rating = group['rating']
    sim_sum = user_sim.sum(axis=0)
    
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    user_prediction_result_df.loc[recipeID] = pred_ratings

  9%|▊         | 30/351 [00:00<00:02, 133.93it/s]

100%|██████████| 351/351 [00:02<00:00, 174.25it/s]


In [62]:
user_prediction_result_df = user_prediction_result_df.transpose()

In [63]:
user_prediction_result_df

,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.922499,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.38492,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.679082,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.237201,0.0,3.417677,1.830127,0.0
990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.44949,0.0,2.129046,1.25,0.0
991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
print(user_prediction_result_df.shape)
print(item_prediction_result_df.shape)

(2645, 351)
(2645, 351)


In [65]:
test_df.head()

,category,recipeID,recipe_name,user_id,rating,review_content
2766,피로회복,441,문어 꽈리고추 조림,1421,3,문어가 없어서 오징어로 대체해봤어요
63,간 건강,779,새우 브로콜리 볶음,1296,5,새우가 냉동실에 많아 다 쏟아붓고했더니 ㅋㅋ 많은양에 양념을 못맞춰 이것더것 조금씩...
3525,호흡기 건강,586,겨울철 가족의 건강을 지켜주는 대추고(차) 만드는법,3177,5,진짜 열심히 만들었는데 만들고 보니 된장 같이 생겼네요 ㅋㅋ 물에 타먹으니 맛있어요 ^^
2126,장 건강,325,느타리버섯양념구이,596,5,맛있어요~ 벌써 여러 번 해 먹었답니다 ^^
25,간 건강,768,생생정보 오이미역냉국 황금레시피,52,5,너무맞있게먹었요.


In [66]:
def evaluate(test_df, prediction_result_df):
    groups_with_recipe_ids = test_df.groupby(by='recipeID')
    groups_with_user_ids = test_df.groupby(by='user_id')
  
    intersection_recipe_ids = sorted(list(set(list(prediction_result_df.columns)).intersection(set(list(groups_with_recipe_ids.indices.keys())))))
    intersection_user_ids = sorted(list(set(list(prediction_result_df.index)).intersection(set(groups_with_user_ids.indices.keys()))))

    print(len(intersection_recipe_ids))
    print(len(intersection_user_ids))
  
    compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_recipe_ids]
  
    grouped = test_df.groupby(by='user_id')
    result_df_list = []  # List to store final_df for each user
  
    for user_id, group in tqdm(grouped):
        if user_id in intersection_user_ids:
            pred_ratings = compressed_prediction_df.loc[user_id][compressed_prediction_df.loc[user_id].index.\
                intersection(list(group['recipeID'].values))]
            pred_ratings = pred_ratings.to_frame(name='rating').reset_index().\
                rename(columns={'index':'recipeID','rating':'pred_rating'})
            actual_ratings = group[['rating', 'recipeID']].rename(columns={'rating':'actual_rating'})
          
            final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['recipeID'])
            final_df = final_df.round(4)
            
            result_df_list.append(final_df)  # Append final_df to result_df_list

    # Concatenate all dataframes in result_df_list
    result_df = pd.concat(result_df_list, ignore_index=True)
    
    return result_df

In [67]:
evaluate(test_df, user_prediction_result_df)

171
69


 86%|████████▌ | 597/695 [00:00<00:00, 906.49it/s]

100%|██████████| 695/695 [00:00<00:00, 919.47it/s]


,actual_rating,recipeID,pred_rating
0,5,73,0.0
1,5,84,3.328633
2,5,613,1.25
3,4,750,0.0
4,5,618,0.0
...,...,...,...
68,5,356,0.0
69,5,743,0.0
70,5,845,0.0
71,5,625,0.0


In [68]:
evaluate(test_df, item_prediction_result_df)

171
69


  8%|▊         | 53/695 [00:00<00:01, 492.05it/s]

100%|██████████| 695/695 [00:00<00:00, 1186.30it/s]


,actual_rating,recipeID,pred_rating
0,5,73,0.0
1,5,84,0.057446
2,5,613,0.081667
3,4,750,0.0
4,5,618,0.0
...,...,...,...
68,5,356,0.0
69,5,743,0.0
70,5,845,0.0
71,5,625,0.0


In [69]:
result_df = evaluate(test_df, user_prediction_result_df)
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

171
69


  0%|          | 0/695 [00:00<?, ?it/s]

100%|██████████| 695/695 [00:00<00:00, 1619.57it/s]

RMSE: 4.462581488251801


In [70]:
result_df = evaluate(test_df, item_prediction_result_df)
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

171
69


100%|██████████| 695/695 [00:00<00:00, 1627.05it/s]

RMSE: 4.852610075163307


In [71]:
non_zero_ratio = user_sparse_matrix.astype(bool).sum(axis=0) / len(user_sparse_matrix)
print("각 열의 0이 아닌 값의 비율:")
print(non_zero_ratio)
# 데이터프레임의 전체 요소 수 계산
total_elements = user_sparse_matrix.size
# 0이 아닌 값의 개수 계산
non_zero_count = user_sparse_matrix.astype(bool).sum().sum()
# 0이 아닌 값의 비율 계산
non_zero_percentage = (non_zero_count / total_elements) * 100
print("0이 아닌 값의 총 비율: {:.2f}%".format(non_zero_percentage))

각 열의 0이 아닌 값의 비율:
recipeID
1      0.001134
2      0.000378
4      0.000378
8      0.000756
9      0.003025
         ...   
865    0.006427
868    0.000756
869    0.031380
870    0.001890
872    0.002268
Length: 351, dtype: float64
0이 아닌 값의 총 비율: 0.30%


In [143]:
def get_collabor(cossim_df, title):
    return cossim_df[title].sort_values(ascending=False)[:10]

In [90]:
item_result = get_collabor(item_cossim_df,1)
item_result

recipeID
1      1.0
605    0.0
618    0.0
617    0.0
616    0.0
Name: 1, dtype: float64

In [107]:
print(data.loc[data['recipeID'] == 1, 'recipe_name'])

1    아삭아삭 연근장아찌 만들기
Name: recipe_name, dtype: object


In [158]:
print(data.loc[data['recipeID'] == 1, 'category'])
print(data.loc[data['recipeID'] == 25])
print(data.loc[data['recipeID'] == 255])
print(data.loc[data['recipeID'] == 170])

1    저칼로리 다이어트
Name: category, dtype: object
    recipeID           recipe_name                                summary  \
25        25  미역줄기볶음 - 간단한 밑반찬 만들기  임산부에게 좋은 미역으로 간단한 밑반찬 미역줄기볶음을 만들어보세요.   

                                      ingredient_name   category  \
25  [재료] 염장미역줄기  300g 양파  1/4개 홍고추  1개 당근  1/4개 [양...  저칼로리 다이어트   

              main_source  
25  [염장미역줄기, 양파, 홍고추, 당근]  
     recipeID     recipe_name  \
255       255  무나물 만드는법 천연소화제   

                                               summary  \
255  평소에 잘 체하는 체질이라서 골골 거리는 편이라 무를 이용한 요리를 자주 해먹어요....   

                                       ingredient_name category main_source  
255  [재료] 무  2개 대파  1개 [양념] 굵은소금  1작은술 식용유  약간 다진마늘...     위 건강     [무, 대파]  
     recipeID         recipe_name  \
170       170  건강 한그릇 요리 - 표고버섯덮밥   

                                               summary  \
170  고기로 맛을 내서 따로 육수를 만들지 않아도 되서 더 좋고 맛있고 영양좋은 표고버섯...   

                                       ingredient_name category  \
170  [재료

In [144]:
user_result = get_collabor(user_cossim_df, '2739')
user_result

user_id
2739    1.000000
2735    0.464991
1522    0.464991
2433    0.464991
2439    0.464991
3224    0.464991
1682    0.462147
153     0.462147
3314    0.462147
3289    0.410997
Name: 2739, dtype: float64

In [145]:
user_result.index

Index(['2739', '2735', '1522', '2433', '2439', '3224', '1682', '153', '3314',
       '3289'],
      dtype='object', name='user_id')

In [152]:
print(data_re.loc[data_re['user_id'] == '2433'])

     category  recipeID           recipe_name user_id  rating  \
2665     치매예방       869  절대 후회없는 깻잎김치 황금레시피^^    2433       5   
2991     혈당조절       618             무생채 만드는 법    2433       5   

                                         review_content  
2665  저는 사흘에 한번 요거 해 먹는 거 같아요. 레시피가 암송될 만하지만 나를 못 미더...  
2991  무생채..이렇게 맛있는 무생채 첨 먹어요...그냥 먹어도 맛있지만 무생채에 시금치나...  


In [149]:
print(data_re.loc[data_re['user_id'] == '768'])
print(data_re.loc[data_re['user_id'] == '110'])
print(data_re.loc[data_re['user_id'] == '869'])
# print(data_re.loc[data_re['user_id'] == '479', 'category'])

     category  recipeID recipe_name user_id  rating  \
3389     혈액순환       479        부추무침     768       5   

                    review_content  
3389  와~~~당장 고기구워야될 맛이에요\r\n최고!!!!  
    category  recipeID recipe_name user_id  rating  \
287     노화방지       648        가지무침     110       5   

                                        review_content  
287  ㅋㅋ 오늘 가지무침 처음으로 햇봣당\r\n근뎅 너무너무 빨갛게 하니까 맛잇꼬\r\n...  
     category  recipeID           recipe_name user_id  rating review_content
2716     치매예방       869  절대 후회없는 깻잎김치 황금레시피^^     869       5         맛잇엉ㅋㅋㅋ


In [72]:
import scipy

In [73]:
def get_svd_prediction(user_item_matrix, k):
    u, s, vh = scipy.sparse.linalg.svds(user_item_matrix.to_numpy(), k=k)
    preds = np.dot(np.dot(u, np.diag(s)), vh)

    preds = pd.DataFrame(preds, columns=user_item_matrix.columns, index=user_item_matrix.index)
    preds = (preds - preds.min()) / (preds.max() - preds.min())

    return preds

In [74]:
predictions = get_svd_prediction(user_sparse_matrix,k=64)

In [75]:
predictions

recipeID,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.386660,0.598328,0.458130,0.114098,0.007451,0.415939,0.858654,0.449238,0.090417,0.558125,...,0.650989,0.540442,0.562611,0.511391,0.484858,0.005738,0.457168,0.001371,0.168555,0.284783
1,0.386660,0.598328,0.458130,0.114098,0.007451,0.415939,0.858654,0.449238,0.090417,0.558125,...,0.650989,0.540442,0.562611,0.511391,0.484858,0.005738,0.457168,0.001371,0.168555,0.284783
10,0.680565,0.598034,0.447544,0.114098,0.007451,0.352675,0.869913,0.461836,0.092352,0.566172,...,0.474037,0.800454,0.523559,0.526693,0.365442,0.005775,0.176253,0.001454,0.087062,0.254437
100,0.385511,0.598101,0.461876,0.114098,0.007450,0.415827,0.858337,0.452221,0.090437,0.556322,...,0.651617,0.540905,0.559384,0.507357,0.486429,0.005780,0.456696,0.001208,0.168882,0.285498
1000,0.413866,0.591662,0.544158,0.114170,0.007451,0.419528,0.853500,0.541919,0.991502,0.497404,...,0.536924,0.605673,0.449907,0.425573,0.476415,0.005755,0.370118,0.001383,0.157741,0.277002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,0.435333,0.616965,0.236086,0.114091,0.007452,0.427609,0.853858,0.244613,0.091708,0.678367,...,0.622612,0.716802,0.766545,0.711079,0.474842,0.005561,0.275975,0.001282,0.419250,0.280614
990,0.283074,0.872277,0.674767,0.114107,0.007451,0.394326,0.047260,0.611774,0.086594,1.000000,...,0.492980,0.413614,0.374778,0.321626,0.498231,0.006209,0.524734,0.001402,0.254356,0.208794
991,0.386660,0.598328,0.458130,0.114098,0.007451,0.415939,0.858654,0.449238,0.090417,0.558125,...,0.650989,0.540442,0.562611,0.511391,0.484858,0.005738,0.457168,0.001371,0.168555,0.284783


In [76]:
def calculate_rmse(predictions, actual_data):
    # 예측값과 실제값을 1차원 배열로 변환
    pred_values = predictions.to_numpy().flatten()
    actual_values = actual_data.to_numpy().flatten()
    
    # 예측값과 실제값 사이의 RMSE 계산
    rmse = np.sqrt(mean_squared_error(actual_values, pred_values))
    
    return rmse

In [77]:
# 예측 모델의 RMSE 계산
rmse = calculate_rmse(predictions, user_sparse_matrix)
print("RMSE:", rmse)

RMSE: 0.48804131237486664


In [78]:
user_id = '1657'
user_recipe_ids = data_re[data_re.user_id == user_id].recipeID
user_recipes = data[data.recipeID.isin(user_recipe_ids)]
print(user_recipes.recipe_name, user_recipes.category)

user_predictions = predictions.loc[user_id].sort_values(ascending=False)
user_predictions = user_predictions[~user_predictions.index.isin(user_recipe_ids)]
user_predictions = user_predictions.head(10)

user_recommendations = data[data.recipeID.isin(user_predictions.index)]
user_recommendations['recommendation_score'] = user_predictions.values
print(user_recommendations.recipe_name, user_recommendations.category)

110    닭가슴살 갈비구이
Name: recipe_name, dtype: object 110    머슬업 다이어트
Name: category, dtype: object
12                      재래김볶음
34                  별미 미역줄기잡채
35                쌈다시마 무침 만들기
91            고소한 카레아몬드 멸치 볶음
114                 황태누룽지 한그릇
369                     쭈꾸미볶음
403     얼큰하고 국물이 시원한 두부국 만드는법
445                  영양듬뿍 버섯밥
524    향긋한 달래양념장에 비벼 먹는 표고버섯밥
856                     현미밥 전
Name: recipe_name, dtype: object 12     저칼로리 다이어트
34     저칼로리 다이어트
35     저칼로리 다이어트
91      머슬업 다이어트
114     머슬업 다이어트
369      스트레스 해소
403      스트레스 해소
445         피로회복
524         혈액순환
856         치매예방
Name: category, dtype: object


C:\Users\SSO\AppData\Local\Temp\ipykernel_24420\745060086.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations['recommendation_score'] = user_predictions.values


In [84]:
data_re.groupby(['user_id']).count().sort_values(by='recipe_name')

,category,recipeID,recipe_name,rating,review_content
user_id,,,,,
0,1,1,1,1,1
3000,1,1,1,1,1
3001,1,1,1,1,1
3002,1,1,1,1,1
3004,1,1,1,1,1
...,...,...,...,...,...
2760,4,4,4,4,4
1473,5,5,5,5,5
3190,6,6,6,6,6


In [85]:
data_re.loc[data_re['user_id']=='2854']

,category,recipeID,recipe_name,user_id,rating,review_content
414,노화방지,706,검은콩자반-말캉말캉부드럽게바로 조려요 ^^,2854,5,콩을 불리지도 않고 해서 잘 익을까 했는데 진짜 살캉하게 잘 익었더라구요~\r\n콩...
1872,위 건강,244,백종원 무나물 만드는법,2854,5,저는 싱거워서 소금을 더 추가했어요~\r\n맛있게 먹겠습니다~\r\n감사해요~
1983,위 건강,278,순두부 버섯들깨탕 만들기,2854,5,순두부가 없어서 연두부 넣었어요~\r\n맛있게 먹겠습니다^^
2077,장 건강,311,팽이버섯 달걀국 끓이기,2854,4,간단하고 맛있어요^^
2217,장 건강,356,"표고버섯덮밥, 덮밥요리 간단 레시피",2854,5,아이가 물똥을 싸서 해줘써요~\r\n잘 먹었으면 좋겠네요^^\r\n감사해요~
2275,저칼로리 다이어트,20,깨순볶음,2854,4,국간장을 적게 넣은다고 했는데 그래도 짜네요~\r\n그래도 제가 조아하는 반찬을 제...
2875,혈당조절,616,배추 된장 들깨무침,2854,5,맛있어요^^
3017,혈당조절,618,무생채 만드는 법,2854,5,전 처음에 살짝 싱거워서 양념을 더 추가해줘써요~\r\n너무 맛있네요~\r\n감사해요^^


In [86]:
data_re.loc[data_re['user_id']=='2739']

,category,recipeID,recipe_name,user_id,rating,review_content
18,간 건강,768,생생정보 오이미역냉국 황금레시피,2739,3,달다는 댓글 참고해서 설탕을 덜 넣어 만들었는데 진짜 맛있었어요! 여름 내 맛있게 ...
470,머슬업 다이어트,73,참치깻잎밥전&참치깻잎치즈밥전,2739,5,맛있었어요^^\r\n바싹 구우니 누룽지 같은 맛도 나고 \r\n좋네요.\r\n아이도...
1109,머슬업 다이어트,110,닭가슴살 갈비구이,2739,4,짜다고 해서 간장 조절했는데\r\n짜긴 하네요ㅜㅜ\r\n덜 짜면 분명 맛있을 것 같...
1798,위 건강,243,두부 스테이크,2739,5,소스가 조금 짠 것 같아서 설탕을 조금 넣었어요!\r\n안넣어도 분명 맛있었을 것 ...
2498,치매예방,828,"견과류 듬뿍 넣은 쫄깃,달콤함 ""두부강정""♪",2739,4,맛있었어요^^\r\n견과류가 들어가니 더 건강식같고 좋네요!\r\n다음엔 더 맛있게...
2611,치매예방,865,초간단 양파볶음,2739,5,사과식초 2/3큰술 넣으니\r\n굴소스 특유의 구린 내(?)와 마요네즈 느끼함이 사...
2677,치매예방,869,절대 후회없는 깻잎김치 황금레시피^^,2739,4,설탕 아주 조금 더하고\r\n양파와 당근 잔뜩 채썰어 넣었어요.\r\n완성된 요리에...
3010,혈당조절,618,무생채 만드는 법,2739,4,냉장고에 오래 방치해서 말랑해져가던 무를 처리했어요 ㅎㅎ\r\n그것도 아주 새콤하고...


In [87]:
data_re.loc[data_re['user_id']=='3190']

,category,recipeID,recipe_name,user_id,rating,review_content
458,머슬업 다이어트,71,황잣으로 끓인 ~ 잣죽 - 딸에게# /죽끓이는 물의 황금비율,3190,5,감사합니다~그런데 설명을 잘 못보고 잣을 첨부터 같이 끓였어요~ 다음엔 더 맛있게 ...
1910,위 건강,245,초간단 연두부반찬,3190,5,매실액도 조금 넣고 아주 맛있게 먹었어요~\r\n간단해서 자주 할 것 같아요~ 감사...
1946,위 건강,276,새우젓으로 넣어 끓인 순두부찌개,3190,5,맛이 담백해서 참 좋았어요.\r\n감사합니다...
2058,장 건강,305,시래기표고버섯밥,3190,5,덕분에 맛있게 잘 먹었어요~ \r\n볶는거 생략하고 전기압력밥솥에 고화력백미로 했는...
3218,혈당조절,628,가정식 스타일 팥죽만드는법,3190,5,아이가 저녁으로 팥죽 먹고 싶다고 해서 끓여줬어요~ 압력밥솥에 끓여서 시간단축되고 ...
3354,혈액순환,479,부추무침,3190,5,감사합니다~ 부추무침 맛나게 해먹었습니다~ 쭈욱~ 이 방법대로 하려구요~~~^^


In [88]:
data_re.loc[data_re['user_id']=='1473']

,category,recipeID,recipe_name,user_id,rating,review_content
6,간 건강,768,생생정보 오이미역냉국 황금레시피,1473,5,날씨가 너무더워서 탕이나 찌개.국같은 애들은 좀 끓이기도 그렇더라구요..워낙 몸에 ...
2636,치매예방,869,절대 후회없는 깻잎김치 황금레시피^^,1473,5,저희슈퍼에서는 깻잎을 많이주고 싸게팔거든요ㅡ항상사면 꼭 반넘게남아서..어쩔까하다가 ...
2788,피로회복,458,버섯 들깨탕 만드는 법,1473,5,신랑이 잇몸치료한다고 잘 못먹어서 보다가 이걸로 식사했어요ㅡ버섯종류를 하나만 했는데...
3120,혈당조절,623,구운애호박 무침,1473,5,오ㅡ간단하게해서 맛있게 먹었네요~한끼 반찬으로 괜찮네요~ 잘먹었습니다:)
3270,혈액순환,479,부추무침,1473,5,와ㅡ레시피 짱 쉬워요^^ 고기굽고 부추무침해서 같이 먹었는데 간이 적당하네요:)


In [157]:
data.loc[data['recipe_name']=='매생이굴국', 'main_source']

36    [매생이, 적당량, 굴, 소금, 참기름, 마늘, 청주, 생강즙]
Name: main_source, dtype: object